In [ ]:
#Import required libraries
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,InputLayer,Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from callbacks import all_callbacks
import plotting
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.regularizers import l1

import importlib

In [ ]:
# Read the data
data = fetch_openml('hls4ml_lhc_jets_hlf') 
X, y = data['data'], data['target']

In [ ]:
# Columns exploration
print(f"Shape of X: {X.shape}")
print(f"Shape of target: {y.shape}")
feature_names = X.columns.tolist()
print("Feature names:", feature_names)
# 2. Check for null values in each column
null_values = X.isnull().sum()
print("Null values in each column:")
print(null_values)

In [ ]:
le = LabelEncoder()
y_le= le.fit_transform(y) # y_le numpy array, shape is 83000
y_cat = to_categorical(y_le, 5)  #y_cat  numpy array, shape is 83000,5
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)


In [ ]:
print(y[25])
y_cat[25]

In [ ]:
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:

model = Sequential()
model.add(InputLayer(batch_input_shape=(None, 16), name='input_1'))
#model.add(Input(shape=(16,), name='input_1')) 
model.add(Dense(64, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu1'))
model.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu2'))
model.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='relu', name='relu3'))
model.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='softmax', name='softmax'))


In [ ]:
train = False
if train:
    adam = Adam(learning_rate=0.0001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    cb = all_callbacks(stop_patience=5,
        lr_factor=0.5,
        lr_patience=2,
        lr_epsilon=0.001,
        lr_cooldown=1,
        lr_minimum=1e-5,
        outputDir='model_1',
    )
    model.fit(
        X_train_norm,
        y_train,
        batch_size=1024,
        epochs=10,
        validation_split=0.25,
        shuffle=True,
        callbacks=cb.callbacks_list,
    )
else:
    from tensorflow.keras.models import load_model

    model = load_model('model_1/KERAS_check_best_model.h5')
    print("Model loaded from disk.")

y_keras = model.predict(X_test_norm)
y_keras.shape
print(y_keras.shape)
print(y_test.shape)
print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))


In [ ]:
"""
from tensorflow.keras.models import load_model
model = load_model('model_1/KERAS_check_best_model.h5')
"""
print(model.input_shape)
import json
model_arch = json.loads(model.to_json())
print(json.dumps(model_arch['config']['layers'][0], indent=2))


In [103]:
import os
import yaml
import hls4ml
# Directory where to save reports
custom_dir = 'custom_data'
config_file = 'custom_config.yaml'
# Full path to the config file
config_file_path = os.path.join(custom_dir, config_file)
# Create directory if not exists
if not os.path.exists(custom_dir):
    os.makedirs(custom_dir)
# Create the empty config file (overwrite if exists)
with open(config_file_path, 'w') as f:
    pass  # just create an empty file


In [ ]:
# Load YAML config file
with open('custom_config.yaml', 'r') as f:
    config = yaml.safe_load(f)
import plotting
#config = hls4ml.utils.config_from_keras_model(model, granularity='model', backend='Vitis')
# Sets the configuration (you can do this by.yaml file), output is dictinary (backend dependent) of configurations
from pprint import pprint 
pprint(config)

In [ ]:
# Convert the model o synthesizable HLS C++ code usin the backend dependent appropriate HLS toolchani
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, backend='Vitis', output_dir='model_1/hls4ml_prj', part='xc7z020clg400-1'
)
# Tutorial part:'xcu250-figd2104-2L-e'
#PYNQ board part:'xc7z020clg400-1'
hls_model.write()

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

In [ ]:
hls_model.compile()
X_test_hls = np.ascontiguousarray(X_test_norm)
y_hls = hls_model.predict(X_test_hls)
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
classes = le.classes_

In [ ]:
classes = le.classes_ 
for i, class_name in enumerate(classes):
    print(f"i={i},class_name={class_name}")


In [ ]:

classes = le.classes_

# Binarize integer labels (should match y_cat)
#y_test_bin = label_binarize(y_le, classes=range(len(classes)))

plt.figure(figsize=(10, 10))

for i, class_name in enumerate(classes):
    y_true = y_test[:, i]
    y_keras_score = y_keras[:, i]
    y_hls_score = y_hls[:, i]

    fpr_keras, tpr_keras, _ = roc_curve(y_true, y_keras_score)
    auc_keras = auc(fpr_keras, tpr_keras)

    fpr_hls, tpr_hls, _ = roc_curve(y_true, y_hls_score)
    auc_hls = auc(fpr_hls, tpr_hls)

    plt.plot(fpr_keras, tpr_keras, label=f'Keras {class_name} (AUC={auc_keras:.3f})')
    plt.plot(fpr_hls, tpr_hls, linestyle='--', label=f'HLS {class_name} (AUC={auc_hls:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass ROC Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


In [ ]:
import os

# Add Vitis HLS path to the environment
os.environ["PATH"] += os.pathsep + "/data/opt/Xilinx/Vitis_HLS/2024.1/bin"
# +"/data/opt/Xilinx/Vivado/2024.1"
# (Optional) Check it's visible to Python
#os.system("which vitis_hls")  # Should print the full path

# Now run the build
report=hls_model.build(csim=False)

In [ ]:
import pandas as pd

r = report['CSynthesisReport']

# Create a DataFrame for resource usage
df = pd.DataFrame({
    'Resource': ['BRAM_18K', 'DSP', 'FF', 'LUT', 'URAM'],
    'Used': [int(r['BRAM_18K']), int(r['DSP']), int(r['FF']), int(r['LUT']), int(r['URAM'])],
    'Available': [int(r['AvailableBRAM_18K']), int(r['AvailableDSP']), int(r['AvailableFF']), int(r['AvailableLUT']), int(r['AvailableURAM'])]
})

df['Utilization (%)'] = (df['Used'] / df['Available'] * 100).round(2)

print(df)


In [ ]:
clock_latency = {
    'Target Clock Period (ns)': r['TargetClockPeriod'],
    'Estimated Clock Period (ns)': r['EstimatedClockPeriod'],
    'Best Latency (cycles)': r['BestLatency'],
    'Worst Latency (cycles)': r['WorstLatency'],
    'Interval Min (cycles)': r['IntervalMin'],
    'Interval Max (cycles)': r['IntervalMax'],
}

df_latency = pd.DataFrame(clock_latency.items(), columns=["Metric", "Value"])
 

In [ ]:

# File path for resource report
resources_report_file_path = os.path.join(report_dir, 'resources_report.tsv')
latency_report_file_path   = os.path.join(report_dir, 'latency_report.tsv')
df.to_csv(resources_report_file_path, sep='\t', index=False)
df_latency.to_csv(latency_report_file_path, sep='\t', index=False)